In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/직전년도_추가 - 복사본.csv")

In [3]:
# 직전년도 제거
df = df[df['구분'] != '직전년도'].reset_index(drop=True)

# 정렬
df = df.sort_values(by=['선수명', 'FA년도']).reset_index(drop=True)

# 비교용 원본 데이터프레임
기준_df = df.copy()

# 선수별 최초 '신규' FA년도
first_fa_by_신규 = 기준_df[기준_df['구분'] == '신규'].groupby('선수명')['FA년도'].min().to_dict()

# 선수별 '자격유지' 중 가장 작은 FA년도 (예외처리용)
min_fa_by_자격유지 = 기준_df[기준_df['구분'] == '자격유지'].groupby('선수명')['FA년도'].min().to_dict()

# 선수별 '재자격' 중 가장 작은 FA년도 (예외처리용)
min_fa_by_재자격 = 기준_df[기준_df['구분'] == '재자격'].groupby('선수명')['FA년도'].min().to_dict()

# 전처리 실행
for i, row in df.iterrows():
    선수명 = row['선수명']
    구분 = row['구분']
    fa_year = row['FA년도']

    # 1. 신규
    if 구분 == '신규' and 선수명 in first_fa_by_신규:
        df.at[i, '선수명'] = f"{선수명}({int(first_fa_by_신규[선수명])})"

    # 2. FA 1년차
    elif 구분 == 'FA 1년차':
        df.at[i, '선수명'] = f"{선수명}({int(fa_year)})"

    # 3. 재자격
    elif 구분 == '재자격':
        후보 = 기준_df[
            (기준_df['선수명'] == 선수명) &
            (기준_df['구분'] == 'FA 1년차') &
            (기준_df['FA년도'] < fa_year)
        ]
        if not 후보.empty:
            최신_FA년도 = 후보['FA년도'].max()
            df.at[i, '선수명'] = f"{선수명}({int(최신_FA년도)})"
        elif 선수명 in min_fa_by_재자격:
            df.at[i, '선수명'] = f"{선수명}({int(min_fa_by_재자격[선수명])})"

    # 4. 자격유지
    elif 구분 == '자격유지':
        후보 = 기준_df[
            (기준_df['선수명'] == 선수명) &
            (기준_df['구분'] == 'FA 1년차') &
            (기준_df['FA년도'] < fa_year)
        ]
        if not 후보.empty:
            최신_FA년도 = 후보['FA년도'].max()
            df.at[i, '선수명'] = f"{선수명}({int(최신_FA년도)})"
        elif 선수명 in min_fa_by_자격유지:
            df.at[i, '선수명'] = f"{선수명}({int(min_fa_by_자격유지[선수명])})"

# 결과 출력
print(df)

     FA년도 구단명        선수명 포지션   투   타  인정년수      구분 FA등급  플레이년도  ...  TEAM 삼진  \
0    2011  롯데  강민호(2011)   C  우투  우타   9.0      신규    A   2010  ...      NaN   
1    2012  롯데  강민호(2011)   C  우투  우타   9.0      신규    A   2011  ...      NaN   
2    2013  롯데  강민호(2011)   C  우투  우타   9.0      신규    A   2012  ...    890.0   
3    2014  롯데  강민호(2011)   C  우투  우타   9.0      신규    A   2013  ...    862.0   
4    2015  롯데  강민호(2015)   C  우투  우타   4.0  FA 1년차    A   2014  ...    964.0   
..    ...  ..        ...  ..  ..  ..   ...     ...  ...    ...  ...      ...   
632  2019  KT  황재균(2019)  3B  우투  우타   4.0     재자격    B   2018  ...   1109.0   
633  2020  KT  황재균(2019)  3B  우투  우타   4.0     재자격    B   2019  ...    975.0   
634  2021  KT  황재균(2019)  3B  우투  우타   4.0     재자격    B   2020  ...   1097.0   
635  2022  KT  황재균(2019)  3B  우투  우타   4.0     재자격    B   2021  ...   1047.0   
636  2023  KT  황재균(2023)  3B  우투  우타   NaN  FA 1년차  NaN   2022  ...   1183.0   

     TEAM 병살타  TEAM 희생플라이  TEAM 평균대비수비득

In [5]:
# 저장
df.to_csv("data/선수 연도표기.csv", index=False)